In [1]:
import numpy as np 
import pandas as pd 
import keras
import os
from keras.models import Model
from keras.layers import Dense
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from sklearn.model_selection import train_test_split

In [2]:
data_generator = ImageDataGenerator(rescale=1.0/255, rotation_range=10, zoom_range=0.15, width_shift_range=0.05, height_shift_range=0.05)

In [3]:
train_generator = data_generator.flow_from_directory("../data/train/", target_size=(224, 224),
                                                          class_mode= 'categorical', batch_size=1)

Found 8000 images belonging to 2 classes.


In [4]:
test_generator = data_generator.flow_from_directory("../data/test/", target_size=(224, 224),
                                                          class_mode= 'categorical', batch_size=1)

Found 1600 images belonging to 2 classes.


In [5]:
vggmodel = VGG16(weights='imagenet', include_top=True)

In [6]:
vggmodel.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
for layers in (vggmodel.layers)[:-1]:
    print(layers)
    layers.trainable = False
vggmodel.layers[-1].trainable = True

In [8]:
X = vggmodel.layers[-2].output

In [9]:
predictions = Dense(2, activation="softmax")(X)

In [10]:
model_final = Model(vggmodel.input, predictions)

In [11]:
model_final.compile(loss = "categorical_crossentropy", optimizer = 'adam', metrics=["accuracy"])

In [12]:
model_final.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [13]:
hist = model_final.fit(train_generator, steps_per_epoch=250, epochs=5)

Epoch 1/5
250/250 [==============================] - 252s 914ms/step - loss: 1.1255 - accuracy: 0.5483
Epoch 2/5
250/250 [==============================] - 217s 870ms/step - loss: 0.7198 - accuracy: 0.7006
Epoch 3/5
250/250 [==============================] - 216s 863ms/step - loss: 0.8353 - accuracy: 0.7627
Epoch 4/5
250/250 [==============================] - 296s 1s/step - loss: 0.7862 - accuracy: 0.7387
Epoch 5/5
250/250 [==============================] - 302s 1s/step - loss: 0.6474 - accuracy: 0.7612


In [14]:
loss, accuracy = model_final.evaluate(test_generator, steps=200)

200/200 [==============================] - 195s 964ms/step - loss: 0.8803 - accuracy: 0.6650
